# $$Empirical \text{ } Finance$$
## Portfolio Returns Assignment: 
### _Stock Split Annoucements_

Rohitashwa Chakraborty _(rc47878)_

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from patsy import dmatrices
import yfinance as yf
from datetime import datetime

import wrds
connection = wrds.Connection()

from tqdm.notebook import tqdm
tqdm.pandas()

%matplotlib inline
plt.style.use("bmh")

Loading library list...
Done


## CRSP Dataset

- [CRSP Daily Stock - Securities (dsf)](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-security-files/daily-stock-file/)
- [CRSP Daily Stock Events - Share Outstanding Information (dse)](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-events/share-outstanding/)
- [CRSP Daily Stock Event -Distribution (dsedist)](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-events/distribution/)

# Table Description

In [2]:
df_stock_splits = connection.raw_sql('''
                    SELECT *
                    FROM crsp.dsedist
                    WHERE facshr >= 1.0
                        AND dclrdt IS NOT NULL
                        AND permno IS NOT NULL
                    FETCH FIRST 10000 ROWS ONLY
                ''',
                date_cols= ['dclrdt', 'exdt', 'rcrddt', 'paydt'])
df_stock_splits = df_stock_splits.drop_duplicates(subset= ['permno','dclrdt']).dropna(subset=['permno','dclrdt','facshr'])
df_stock_splits

,permno,distcd,divamt,facpr,facshr,dclrdt,exdt,rcrddt,paydt,acperm,accomp,permco,compno,issuno,hexcd,hsiccd,cusip
0,10001.0,5523.0,0.0,1.0,1.0,1994-05-18,1994-06-27,1994-05-31,1994-06-24,0.0,0.0,7953.0,60007906.0,10398.0,2.0,4925.0,36720410
1,10006.0,5523.0,0.0,1.0,1.0,1963-05-03,1963-09-25,1963-09-03,1963-09-24,0.0,0.0,22156.0,0.0,0.0,1.0,3743.0,00080010
2,10006.0,5523.0,0.0,1.0,1.0,1965-05-07,1965-09-21,1965-08-30,1965-09-20,0.0,0.0,22156.0,0.0,0.0,1.0,3743.0,00080010
3,10009.0,5523.0,0.0,1.0,1.0,1995-07-31,1995-09-01,1995-08-10,1995-08-31,0.0,0.0,7965.0,60007918.0,10416.0,3.0,6030.0,46334710
4,10016.0,5523.0,0.0,2.0,2.0,1989-11-17,1990-01-23,1990-01-15,1990-01-22,0.0,0.0,1728.0,70050056.0,2217.0,3.0,3812.0,81002230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8558,93356.0,5523.0,0.0,1.0,1.0,2019-07-25,2019-08-23,2019-08-08,2019-08-22,0.0,0.0,53399.0,60069381.0,65542.0,3.0,9999.0,78463M10
8559,93423.0,5523.0,0.0,1.0,1.0,2011-05-05,2011-06-28,2011-06-15,2011-06-27,0.0,0.0,53440.0,0.0,0.0,1.0,7996.0,83001A10
8560,93423.0,5523.0,0.0,1.0,1.0,2013-05-08,2013-06-27,2013-06-12,2013-06-26,0.0,0.0,53440.0,0.0,0.0,1.0,7996.0,83001A10
8561,93425.0,5523.0,0.0,1.0,1.0,2013-08-12,2013-08-29,2013-08-26,2013-08-28,0.0,0.0,53442.0,0.0,0.0,4.0,6221.0,91167Q10


In [3]:
df_stock_splits = df_stock_splits.set_index(['permno','dclrdt'])[['facshr']]
df_stock_splits = df_stock_splits.dropna()
df_stock_splits

facshr
permno  dclrdt            
10001.0 1994-05-18     1.0
10006.0 1963-05-03     1.0
        1965-05-07     1.0
10009.0 1995-07-31     1.0
10016.0 1989-11-17     2.0
...                    ...
93356.0 2019-07-25     1.0
93423.0 2011-05-05     1.0
        2013-05-08     1.0
93425.0 2013-08-12     1.0
93436.0 2020-08-10     4.0

[8563 rows x 1 columns]

In [4]:
df_stock_daily = connection.raw_sql('''
                    SELECT dsf.permno, date, dclrdt, ret, shrout, facshr, prc, crsp.dsf.date - stock_split.dclrdt AS date_diff
                    FROM crsp.dsf
                    LEFT JOIN
                    (
                        SELECT permno, dclrdt, facshr
                        FROM crsp.dsedist
                        WHERE facshr >= 1.0
                            AND dclrdt IS NOT NULL
                            AND permno IS NOT NULL
                    ) stock_split ON stock_split.permno = crsp.dsf.permno 
                    WHERE facshr IS NOT NULL
                        AND crsp.dsf.date - stock_split.dclrdt <=  2
                        AND crsp.dsf.date - stock_split.dclrdt >=  0
                    FETCH FIRST 1000 ROWS ONLY
                ''',
                date_cols= ['date', 'rcrddt', 'trtsendt', 'shrenddt'])
df_stock_daily

,permno,date,dclrdt,ret,shrout,facshr,prc,date_diff
0,10001.0,1994-05-18,1994-05-18,-0.016667,1091.0,1.0,14.750,0
1,10001.0,1994-05-19,1994-05-18,0.033898,1091.0,1.0,15.250,1
2,10001.0,1994-05-20,1994-05-18,0.000000,1091.0,1.0,15.250,2
3,10006.0,1963-05-03,1963-05-03,0.005525,1453.0,1.0,91.000,0
4,10006.0,1965-05-07,1965-05-07,-0.001357,2968.0,1.0,92.000,0
...,...,...,...,...,...,...,...,...
995,11628.0,1988-03-11,1988-03-09,0.039394,896.0,1.0,42.875,2
996,11628.0,1989-01-18,1989-01-18,0.000000,2009.0,1.0,-34.750,0
997,11628.0,1989-01-19,1989-01-18,0.000000,2009.0,1.0,-34.750,1
998,11628.0,1989-01-20,1989-01-18,0.000000,2009.0,1.0,-34.750,2


In [5]:
df_stock_daily = df_stock_daily.set_index(['permno','date'])[['ret','shrout','prc']]
df_stock_daily

ret  shrout     prc
permno  date                                
10001.0 1994-05-18 -0.016667  1091.0  14.750
        1994-05-19  0.033898  1091.0  15.250
        1994-05-20  0.000000  1091.0  15.250
10006.0 1963-05-03  0.005525  1453.0  91.000
        1965-05-07 -0.001357  2968.0  92.000
...                      ...     ...     ...
11628.0 1988-03-11  0.039394   896.0  42.875
        1989-01-18  0.000000  2009.0 -34.750
        1989-01-19  0.000000  2009.0 -34.750
        1989-01-20  0.000000  2009.0 -34.750
        1997-10-08 -0.005882  5275.0  42.250

[1000 rows x 3 columns]

In [6]:
df_join = pd.merge(df_stock_daily.reset_index(), 
                    df_stock_splits.reset_index(), 
                    how= 'left',
                    left_on = ['permno','date'],
                    right_on = ['permno','dclrdt'])[['permno','date','ret', 'prc','facshr','shrout']]
df_join = df_join.set_index(['permno','date'])
df_join

ret     prc  facshr  shrout
permno  date                                        
10001.0 1994-05-18 -0.016667  14.750     1.0  1091.0
        1994-05-19  0.033898  15.250     NaN  1091.0
        1994-05-20  0.000000  15.250     NaN  1091.0
10006.0 1963-05-03  0.005525  91.000     1.0  1453.0
        1965-05-07 -0.001357  92.000     1.0  2968.0
...                      ...     ...     ...     ...
11628.0 1988-03-11  0.039394  42.875     NaN   896.0
        1989-01-18  0.000000 -34.750     1.0  2009.0
        1989-01-19  0.000000 -34.750     NaN  2009.0
        1989-01-20  0.000000 -34.750     NaN  2009.0
        1997-10-08 -0.005882  42.250     1.0  5275.0

[1000 rows x 4 columns]

## Compustat
comp/secd

<table id="data-table" class="table table-bordered table-hover table-striped table-mobile tablesorter tablesorter-default">
<thead>
<tr>
<th class="text-nowrap">Variable Name</th>
<th>Type</th>
<th>Length</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td style="background-color: inherit;">adrrc</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">ADR Ratio - Daily</td>
</tr>
<tr>
<td style="background-color: inherit;">ajexdi</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">Adjustment Factor (Issue)-Cumulative by Ex-Date</td>
</tr>
<tr>
<td style="background-color: inherit;">
anncdate
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/anncdate/&quot;);">
<span class="fa fa-question-circle float-right"></span>
</a>
</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Dividend Declaration Date
</td>
</tr>
<tr>
<td style="background-color: inherit;">
capgn
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/capgn/&quot;);">
<span class="fa fa-question-circle float-right"></span>
</a>
</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Capital Gains - Daily
</td>
</tr>
<tr>
<td style="background-color: inherit;">
capgnpaydate
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/capgnpaydate/&quot;);">
<span class="fa fa-question-circle float-right"></span>
</a>
</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Capital Gains Payment Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cheqv


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cheqv/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Cash Equivalent Distributions
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cheqvpaydate
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cheqvpaydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>
</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Cash Equivalent Distributions per Share Payment Date
</td>
</tr>
<tr>
<td style="background-color: inherit;">
cik
<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cik/&quot;);"><span class="fa fa-question-circle float-right"></span></a>
</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">10</td>
<td style="background-color: inherit;">
CIK Number
</td>
</tr>

<tr>
<td style="background-color: inherit;">
conm


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/conm/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">57</td>
<td style="background-color: inherit;">
Company Name
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cshoc


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cshoc/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Shares Outstanding
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cshtrd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cshtrd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Trading Volume - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
curcdd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/curcdd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">3</td>
<td style="background-color: inherit;">
ISO Currency Code - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
curcddv


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/curcddv/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">3</td>
<td style="background-color: inherit;">
ISO Currency Code - Dividend
</td>
</tr>

<tr>
<td style="background-color: inherit;">
cusip


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/cusip/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">9</td>
<td style="background-color: inherit;">
CUSIP
</td>
</tr>

<tr>
<td style="background-color: inherit;">
datadate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/datadate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Data Date - Dividends
</td>
</tr>

<tr>
<td style="background-color: inherit;">
div


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/div/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Dividends per Share - Ex Date - Daily (Issue)
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Cash Dividends - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divdpaydate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divdpaydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Cash Dividends - Daily Payment Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divdpaydateind


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divdpaydateind/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">1</td>
<td style="background-color: inherit;">
Cash Dividends - Daily Payment Date Indicator
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divrc


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divrc/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
None
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divrcpaydate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divrcpaydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
None
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divsp


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divsp/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Special Cash Dividends - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
divsppaydate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/divsppaydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Special Cash Dividends - Daily Payment Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
dvi


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/dvi/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Indicated Annual Dividend - Current
</td>
</tr>

<tr>
<td style="background-color: inherit;">
dvrated


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/dvrated/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Indicated Annual Dividend Rate - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
eps


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/eps/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Current EPS
</td>
</tr>

<tr>
<td style="background-color: inherit;">
epsmo


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/epsmo/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Current EPS Month
</td>
</tr>

<tr>
<td style="background-color: inherit;">
exchg


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/exchg/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Stock Exchange Code
</td>
</tr>

<tr>
<td style="background-color: inherit;">
fic


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/fic/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">3</td>
<td style="background-color: inherit;">
Current ISO Country Code - Incorporation
</td>
</tr>

<tr>
<td style="background-color: inherit;">
gvkey


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/gvkey/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">6</td>
<td style="background-color: inherit;">
Global Company Key - Dividends
</td>
</tr>

<tr>
<td style="background-color: inherit;">
iid


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/iid/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">3</td>
<td style="background-color: inherit;">
Issue ID - Dividends
</td>
</tr>

<tr>
<td style="background-color: inherit;">
paydate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/paydate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Dividend Payment Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
paydateind


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/paydateind/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">1</td>
<td style="background-color: inherit;">
Dividend Payment Date Indicator
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prccd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prccd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price - Close - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prchd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prchd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price - High - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prcld


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prcld/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price - Low - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prcod


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prcod/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price - Open - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
prcstd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/prcstd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Price Status Code - Daily
</td>
</tr>

<tr>
<td style="background-color: inherit;">
recorddate


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/recorddate/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">date</td>
<td style="background-color: inherit;"></td>
<td style="background-color: inherit;">
Dividend Record Date
</td>
</tr>

<tr>
<td style="background-color: inherit;">
secstat


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/secstat/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">1</td>
<td style="background-color: inherit;">
Security Status Marker
</td>
</tr>

<tr>
<td style="background-color: inherit;">
tic


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/tic/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">8</td>
<td style="background-color: inherit;">
Ticker Symbol
</td>
</tr>

<tr>
<td style="background-color: inherit;">
tpci


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/tpci/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">string</td>
<td style="background-color: inherit;">8</td>
<td style="background-color: inherit;">
Issue Type Code
</td>
</tr>

<tr>
<td style="background-color: inherit;">
trfd


<a href="javascript:showMe(&quot;/data-dictionary/comp_na_daily_all/secd/trfd/&quot;);"><span class="fa fa-question-circle float-right"></span></a>


</td>
<td style="background-color: inherit;">double</td>
<td style="background-color: inherit;">53</td>
<td style="background-color: inherit;">
Daily Total Return Factor
</td>
</tr>

</tbody>

</table>

In [7]:
corpus_df = connection.raw_sql(
            '''
                SELECT conm, MAX(anncdate) AS last_divident_announced
                FROM (
                    SELECT *
                    FROM comp.secd
                    WHERE anncdate IS NOT NULL
                    ORDER BY anncdate DESC
                    FETCH FIRST 1000 ROWS ONLY
                ) t
                GROUP BY conm
                ORDER BY last_divident_announced
                FETCH FIRST 10000 ROWS ONLY
'''
        )
corpus_df

,conm,last_divident_announced
0,RED ROCK RESORTS INC,2021-11-10
1,MEGA WORLD CORP,2021-11-10
2,RADIAN GROUP INC,2021-11-10
3,POWER FINANCIAL CORP,2021-11-10
4,BANCO DO BRASIL,2021-11-10
...,...,...
801,MACKENZIE AGGR BO INDEX ETF,2021-11-25
802,MCKNZIE CND SH TR FX INC ETF,2021-11-25
803,MACKENZIE DE BOND INDEX ETF,2021-11-25
804,MACKENZIE EM MKTS BOND INDEX,2021-11-25
